In [ ]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.circuit import Parameter 



import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
#https://stackoverflow.com/questions/9600801/evenly-distributing-n-points-on-a-sphere
#Create n evenly distributed points on a sphere

num_pts = 10000
indices = np.arange(0, num_pts, dtype=float) + 0.5

theta = np.arccos(1 - 2*indices/num_pts)
phi = (np.pi * (1 + 5**0.5) * indices)%(2*np.pi)
coords=[]

for theta_elem,phi_elem in zip(theta,phi):
    coords.append((theta_elem,phi_elem))

x, y, z = np.cos(phi) * np.sin(theta), np.sin(phi) * np.sin(theta), np.cos(theta);

# Plot the surface.
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, template="plotly_white")

fig.show()
target_points=np.array(coords)
# save the data, optional
np.savetxt("target_points_ibmq_athens.csv",target_points)

In [ ]:
theta_param, phi_param = Parameter('theta_param'), Parameter('phi_param') 

qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(3, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.ry(theta_param, qreg_q[0])
circuit.rz(phi_param, qreg_q[0])
    
circuit.ry(1.107149, qreg_q[1])
circuit.cx(qreg_q[1], qreg_q[2])
circuit.ry(0.729728, qreg_q[2])
circuit.cx(qreg_q[2], qreg_q[1])
circuit.ry(0.463648, qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[2])
circuit.cx(qreg_q[1], qreg_q[0])
circuit.cx(qreg_q[2], qreg_q[0])

circuit.rz(-phi_param, qreg_q[0])
circuit.ry(-theta_param, qreg_q[0])
circuit.rz(-phi_param, qreg_q[1])
circuit.ry(-theta_param, qreg_q[1])

circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[1])

In [ ]:
IBMQ.save_account('YOUR_TOKEN')

N_shots=8192
provider = IBMQ.load_account()

#backend = least_busy(provider.backends(simulator=False))
#print(backend)
backend = provider.backends.ibmq_athens

In [ ]:
runningjobs=[]
job_ids=[]
index=130

while index*75 < num_pts:
    if((index+1)*75<num_pts):
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:(index+1)*75]]
        qobj = assemble(transpile(circuits, backend=backend, optimization_level=0), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_athens.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    else:
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:]]
        qobj = assemble(transpile(circuits, backend=backend, optimization_level=0), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_athens.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    index=index+1

In [ ]:
print(index)

In [ ]:
read_job_ids = open("job_ids_athens.txt").read().splitlines()
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    #test both the copies
    marg_prob0_qubit0=job["000"]/N_shots+job["010"]/N_shots #calculate the marginal distribution
    marg_prob0_qubit1=job["000"]/N_shots+job["001"]/N_shots #calculate the marginal distribution
    marg_prob1_qubit0=1-marg_prob0_qubit0
    marg_prob1_qubit1=1-marg_prob0_qubit1
    results_probabilities.append([marg_prob0_qubit0,marg_prob1_qubit0, marg_prob0_qubit1, marg_prob1_qubit1])
    
results_probabilities=np.array(results_probabilities)

In [ ]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [ ]:
data_sheet.head()

In [ ]:
for (point,probabilities) in zip(target_points,results_probabilities):
    with open('results_ibm_athens.txt', 'a') as file:
        file.write(str(point[0])+"\t"+str(point[1])+"\t"+str(probabilities[0])+"\t"+str(probabilities[1])+"\t"+
                   str(probabilities[2])+"\t"+str(probabilities[3])+"\n")

In [ ]:
# save to excel, optional
data_sheet.to_excel("data_sheet_ibm_athens.xlsx")

In [ ]:
data_sheet=pd.read_excel("data_sheet_ibm_athens.xlsx")

In [ ]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0/(5/6), template="plotly_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

In [ ]:
#FIDELITY OF SECOND COPY
# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_1/(5/6), template="plotly_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_1))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_1))

In [ ]:
#CHECK SYMMETRY, I.E. PLOT fidelity of qubit 0/fidelity of qubit 2

df = px.data.iris()
ratio=data_sheet.prob_0_qubit_0/data_sheet.prob_0_qubit_1
fig = px.scatter_3d(df, x, y, z, color=ratio, template="simple_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(ratio))
print("Standard deviation: ", np.std(ratio))